## Factorization Machines
Factorization Machines have been described as state of the art for many
recommendation systems. Yet, experience has shown these models to suffer from slow
training and local minima. Use a large(ish) dataset and characterize where FMs are easy
to fit and accurate and where they are not.
1. Start with models that have no side information, and are only user and item ratings.
Specifically, subsample datasets from small to large, and subsample users/items
from sparsely-populated to well-populated, and train and test FMs. Where do they
work the best? Where do they fail? Can you set good rules of thumbs for their
training and use?
2. Next use side information about users or items. Answer the same questions as
above.

In [23]:
import pandas as pd
import numpy as np
import pandasql as ps

pd.set_option('display.max_rows', 10)

In [21]:
anime = pd.read_csv('anime.csv')
rating = pd.read_csv('rating.csv')